Preface: I used Python 3 (v 5.5.0) to write this project.

# Simulating and Compressing Data

In this notebook, I used ```numpy``` to generate random data and ```gzip```/```bzip2```/```pbzip2```/```ArithmeticCompress``` to compress this data. 

A general outline of this lab:
1. Simulating random binary, nucleotide, and protein data
2. Compress random data
3. Questions about compression
4. Compressing HIV gp120 homolog data
5. Estimations for larger datasets (1000 TB)

## 1. Simulating Random Data

I first generated 100 MB of random binary data for each set 100%, 90%, 80%, 70%, 60%, and 50% zeros. Output in bits.

In [1]:
# 100 MB of random data containing 100% zeros

import numpy as np

Hundred = np.random.choice([0, 1], size=[838860800], replace=True, p=[1.0, 0.0])
Hundred = np.packbits(Hundred)
print(Hundred)
open("100_Zeros", "wb").write(Hundred)

[0 0 0 ... 0 0 0]


104857600

In [2]:
# 100 MB of random data containing 90% zeros

import numpy as np

Ninety = np.random.choice([0, 1], size=[838860800], replace=True, p=[0.9, 0.1])
Ninety = np.packbits(Ninety)
print(Ninety)
open("90_Zeros", "wb").write(Ninety)

[ 64  32   0 ...   1   4 136]


104857600

In [3]:
# 100 MB of random data containing 80% zeros

import numpy as np

Eighty = np.random.choice([0, 1], size=[838860800], replace=True, p=[0.8, 0.2])
Eighty = np.packbits(Eighty)
print(Eighty)
open("80_Zeros", "wb").write(Eighty)

[ 42 170 128 ...  65  33  65]


104857600

In [4]:
# 100 MB of random data containing 70% zeros

import numpy as np

Seventy = np.random.choice([0, 1], size=[838860800], replace=True, p=[0.7, 0.3])
Seventy = np.packbits(Seventy)
print(Seventy)
open("70_Zeros", "wb").write(Seventy)

[12 96  2 ... 16 51 14]


104857600

In [5]:
# 100 MB of random data containing 60% zeros

import numpy as np

Sixty = np.random.choice([0, 1], size=[838860800], replace=True, p=[0.6, 0.4])
Sixty = np.packbits(Sixty)
print(Sixty)
open("60_Zeros", "wb").write(Sixty)

[ 32   7 186 ...  49  96 168]


104857600

In [6]:
# 100 MB of random data containing 50% zeros

import numpy as np

Fifty = np.random.choice([0, 1], size=[838860800], replace=True, p=[0.5, 0.5])
Fifty = np.packbits(Fifty)
print(Fifty)
open("50_Zeros", "wb").write(Fifty)

[188 161  95 ...  73  20 241]


104857600

Next, I generated random DNA and protein data (100 million letters long) and saved them as FASTA files.

In [7]:
# 100 million random nucleotide sequence

DNA = np.random.choice(['A', 'T', 'G', 'C'], size=100000000, replace=True, p=[0.25, 0.25, 0.25, 0.25])
print(DNA)
open("nt_seq.fa", "w").write("".join(DNA))

['A' 'T' 'G' ... 'C' 'T' 'A']


100000000

In [8]:
# 100 million random amino acid sequence

Pro = np.random.choice(['G','A','V','L','I','D','E','N','Q','P','F','W','K','C','M','Y','R','H','S','T'], 
                       size = 100000000, replace=True, p=[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
                                                          0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
                                                          0.05, 0.05, ])
print(Pro)
open("pro_seq.fa", "w").write("".join(Pro))

['Q' 'A' 'I' ... 'V' 'Q' 'C']


100000000

## 2. Compressing Random Data

I used **gzip**, **bzip2**, **pbzip2**, and **ArithmeticCompress** on **Terminal** to compress the random data sets generated in Part 1.

The ```time``` command reported how long each operation took. For each data set, I used the series of commands

```
time gzip -k Filename
time bzip2 -k Filename
time pbzip2 -k Filename
time ArithmeticCompress Filename Filename.art
```

```time``` reports ```real```, ```sys```, and ```user``` times. The table below only reports ```real``` time for each operation. "Arith" is short for ArithmeticCompress.

### Results

<img src="Part 2 Table.jpg" width="60%">

## 3. Questions about Compression

_Which algorithm achieves the best level of compression on each file type?_  
```ArithmeticCompress``` compresses text files (100_Zeros, ..., 50_Zeros) and FASTA files best but also takes the longest time to complete.

_Which algorithm is the fastest?_  
```pbzip2``` compresses my data the fastest.

_What is the difference between_ ```bzip2``` _and_ ```pbzip2```_? Do you expect one to be faster and why?_  
```pbzip2``` is the parallel version of ```bzip2```. This means that ```pbzip2``` runs numerous ```bzip2``` algorithms simultaneously to compress files of interest. As such, it is quite clear that ```pbzip2``` is faster than ```bzip2```.

_How does the level of compression change as the percentage of zeros increases? Why does this happen?_  
Compression increases with increasing percentage of zeros. This is because as percentage of zeros increases, there is less "unique" data in the file. There is more statistical redundancy with increasing percentage of zeros, making compression easier.

_What is the minimum number of bits required to store a single DNA base?_  
2 bits, as log<sub>2</sub>(4) = 2

_What is the minimum number of bits required to store a single amino acid letter?_  
5 bits, as log<sub>2</sub>(20) ≈ 4.322

_In your tests, how many bits did ```gzip``` and ```bzip2``` actually require to store your random DNA and protein sequences?_  
```gzip``` required 29.2 MB and 60.6 MB for the random DNA and protein sequences, respectively. ```bzip2``` required 27.3 MB and 55.3 MB for the same random DNA and protein sequences, respectively. 

_Are ```gzip``` and ```bzip2``` performing well on DNA and proteins?  
Yes, considering the original filesize for the random DNA and protein sequences were both 100 MB. It seems that ```bzip2``` is slightly better for compressing the data, though it does take longer than ```gzip``` for more random data.

## 4. Compressing Real Data

We now move on to interesting data. For this lab, I used **NCBI Nucleotide** to find 10 different HIV gp120 homologs. I concatenated their DNA sequences into a single multi-FASTA file ```HIV_gp120.fa```: GenBank AF236860.1, AF236859.1, AJ429917.1, AJ429906.1, AJ429920.1, AJ429919.1, AJ429915.1, AJ429913.1, AJ429909.1, and AJ429908.1.

_A priori, do you expect to achieve better or worse compression here than random data? Why?_

I expect better compression because biological data is _not random_. There are regions of nucleotide and protein sequences that have repeats (e.g. GC rich regions for higher DNA melt temperatures in thermophilic microbes) that are evolutionarily selected for. These repeats, along with other structural features, are statistically redundant and thus easier to compress.

### Compressing the multi-FASTA using ```gzip```, ```bzip2```, and ```ArithmeticCompress```

Like Part 2, I used **gzip**, **bzip2**, and **ArithmeticCompress** on **Terminal** using the commands:

```
time gzip -k HIV_gp120.fa
time bzip2 -k HIV_gp120.fa
time ArithmeticCompress HIV_gp120.fa HIV_gp120.fa.art
```

#### Results

<img src="Part 4 Table.jpg" width="50%">

_How does the compression ratio of this data compare to random data?_  

The original filesize for ```HIV_gp120.fa``` is 12.4 kB. As such, the ```gzip```, ```bzip2```, and ```ArithmeticCompress``` compression ratios (compressed size divided by original size) are 25.2%, 25.2%, and 40.6%, respectively.

Excluding the 100_Zeros random binary data, the compression ratio for this data is better than that of the random binary data (all >50%). Interestingly, the compression ratio for this data is better than the random DNA file when using ```gzip``` and ```bzip2```, though by a small margin (~4%).

## 5. Estimations for Larger Datasets



_Parameters_: 1000 TB of data is generated everyday. 80% of this data are genomes/plasmids with significant similarities, 10% are protein sequences, and 10% are random binary microscope images. I will earn a $500 bonus for every 1% reduction in data (10 TB), per day. Benchmark data for calculations referenced from Part 1.

For genomes and plasmids, I would use ```pbzip2``` given its fast speed and relatively efficient compression of random DNA sequence data. The compression ratio for the random DNA FASTA file was 27.3 percent, with 100 MB compressed to 27.3 MB in 0.682s. We can thus compress 12.67 TB to 3.46 TB daily. **9.21 TB saved and $460.50 earned per day.**

For protein sequences, I would use ```pbzip2``` for the same reasons as for genomes and plasmids. The compression ratio for the random protein FASTA file was 55.3 percent, and 100 MB was compressed to 55.3 MB in 0.802s. We can thus compress 10.77 TB to 5.96 TB daily. **4.81 TB saved and $240.50 earned per day.**

Assuming the binary microscope images are completely random, compression is not possible. Complete randomness implies equal probability of ```0``` and ```1```, of which is modeled by the 50_Zeros benchmark data set.

Using these algorithms, I will save 5117.3 TB (**1.402 percent of total data**) and earn $255865.00 per annum. Pretty good for some simple algorithms!

These calculations are not accurate because they are premised on lackluster data, with error propagating quickly (the benchmark data are not representative biological data). Furthermore, different files have different compressibilities and these calculations assume only one operation is run per time interval. For an enterprise that generates 1000 TB, we would expect numerous computers to run numerous operations simultaneously. This makes it difficult to use my benchmark data to make any concrete conclusions.